In [1]:
import icalendar
import urllib
import time

from datetime import date, datetime, timedelta
from pytz import utc, timezone

# diese module helfen und beim handling der recurrence rules
from dateutil.rrule import *
from dateutil.parser import *

# wir wollen nur events anzeigen, die zwischen now und then liegen
now = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
then = now + timedelta(21) #21 in der Zukunft

# hier speichern wir alle events ab, die wir anzeigen wollen
data = []
fmt = "%d.%m.%Y %H:%M"
timezoneEF = timezone('Europe/Berlin')

In [2]:
# wir ziehen uns den bytespeicher kalender rein
f = urllib.urlopen('http://www.google.com/calendar/ical/2eskb61g20prl65k2qd01uktis%40group.calendar.google.com/public/basic.ics')
cal = icalendar.Calendar.from_ical(f.read())


In [3]:
# über alle VEVENT interieren
for ev in cal.walk('VEVENT'):
    # ein gescheites VEVENT hat einen Startzeitpunkt
    start = icalendar.prop.vDDDTypes.from_ical(ev["DTSTART"])
    
    # wir unterscheiden:
    # * Events kürzer als einen Tag
    # * Events über einen ganzen Tag
    
    # ist der Startzeitpunkt ein Datetime, haben wir ein Event kürzer als einen Tag
    if isinstance(start, datetime):
        
        rset = rruleset()
        info = ""
        loc = ""
        
        if "SUMMARY" in ev:
            info =  ev["SUMMARY"]
        else:
            continue # events ohne summary zeigen wir nicht an!
        
        if "LOCATION" in ev:
            loc = ev["LOCATION"]
        else:
            loc = "Liebknechtstrasse 8"

        
        # wiederholungen
        if "RRULE" in ev: 
            
            #start und rule extrahieren
            #start und rule in ein ruleset geben 
            #und alle vorkommen zwischen now und then finden
            ical_dtstart = (ev.get("DTSTART")).to_ical()
            ical_rrule = (ev.get('RRULE')).to_ical()
            rset.rrule(rrulestr(ical_rrule, dtstart=parse(ical_dtstart),ignoretz=1))
            
            #TODO hier noch ein ausnahmen-ruleset einbauen (kvh)
            
            #alle wiederholungen die zwischen now und then liegen merken
            #ACHTUNG hier wird irgendwie schon korrekt von utc nach local umgerechnet
            for e in rset.between(now, then):
                data.append({
                'datetime': e.strftime(fmt),
                'datetime_sort': e.strftime(fmt),
                'info': info,
                'loc' : loc,
                })
        
        # keine wiederholungen
        else: 
            #alle wiederholungen merken die zwischen now und then liegen
            #ACHTUNG hier muessen wir noch von hand von utc nach local umrechnen
            if start < utc.localize(now) or start > utc.localize(then):
                    continue
            
            data.append({
                'datetime': start.astimezone(timezoneEF).strftime(fmt),
                'datetime_sort': start.astimezone(timezoneEF).strftime(fmt),
                'info': info,
                'loc' : loc,
                })

#übelst komplizierte "sortiere-hash-nach-datumsspalte" Funktion
data = sorted(data,
    key=lambda k: time.mktime(datetime.strptime(
    k['datetime_sort'], "%d.%m.%Y %H:%M").timetuple()))            


for d in data:
    print "  %s - %s (%s)" % (d['datetime'], d['info'], d['loc'])
                
                
            #print mytemp
            #gaga = list(rrulestr(ical_rrule, dtstart=parse(ical_dtstart)))
            #print "---- %d/%d/%d %d:%d" % (gaga[1].year, gaga[1].month, gaga[1].day, gaga[1].hour, gaga[1].minute)
            #rrule = ev.get('RRULE')
            #rset.rrule(rrule.to_ical())
            #print rset[1]
            
        
        #print " Dtstart %d/%d/%d %d:%d" % (start.year, start.month, start.day, start.hour, start.minute)
    
    #ganztags events
    #elif isinstance(start, date):
       
        #if "SUMMARY" in ev:
            #print "Summary: %s" % ev["SUMMARY"]
        
        #wiederholungen
        #if "RRULE" in ev:
            #if "FREQ" in ev["RRULE"]:
                #print "  Frequency: %s" % ev["RRULE"]["FREQ"][0]
                
        #print " %d/%d/%d" % (start.year, start.month, start.day)        
        
        
    
    
    

  24.11.2015 18:00 - Spieleentwicklung mit Unity (FSU Jena, Informatik Fakultät, Seminarraum 3319, Ernst-Abbe-Platz, Jena)
  25.11.2015 19:00 - Bytespeicher am Mittwoch (Bytespeicher, Liebknechtstraße 8, 99085 Erfurt, Deutschland)
  26.11.2015 19:00 - SDR Workshop - Rauschen (Bytespeicher, Liebknechtstraße 8, 99085 Erfurt, Deutschland)
  02.12.2015 19:00 - Bytespeicher am Mittwoch (Bytespeicher, Liebknechtstraße 8, 99085 Erfurt, Deutschland)
  02.12.2015 20:00 - Freifunk Kennenlerntreffen (Bytespeicher, Liebknechtstraße 8, 99085 Erfurt, Deutschland)
  05.12.2015 15:00 - Liquid Erfurt Workshop (Bytespeicher, Liebknechtstraße 8, 99085 Erfurt, Deutschland)
  08.12.2015 19:00 - Freifunk Arbeitstreffen (Bytespeicher, Liebknechtstraße 8, 99085 Erfurt, Deutschland)
  09.12.2015 19:00 - Bytespeicher am Mittwoch (Bytespeicher, Liebknechtstraße 8, 99085 Erfurt, Deutschland)
